In [1]:
!pip install rpy2==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 KB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp38-cp38-linux_x86_64.whl size=318379 sha256=17fdbfd905a42cfa9a3beab95a6255c248d704be3efe1fb010f6f110b12c26b2
  Stored in directory: /root/.cache/pip/wheels/6b/40/7d/f63e87fd83e8b99ee837c8e3489081c4b3489134bc520235ed
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


In [2]:
%load_ext rpy2.ipython

### **Logistic Regression and GLM:**

#### Read the dataset into variable df

In [3]:
%%R
df=read.csv('/content/sample_data/airline_passenger_satisfaction.csv')
head(df)

  ID Gender Age Customer.Type Type.of.Travel    Class Flight.Distance
1  1   Male  48    First-time       Business Business             821
2  2 Female  35     Returning       Business Business             821
3  3   Male  41     Returning       Business Business             853
4  4   Male  50     Returning       Business Business            1905
5  5 Female  49     Returning       Business Business            3470
6  6   Male  43     Returning       Business Business            3788
  Departure.Delay Arrival.Delay Departure.and.Arrival.Time.Convenience
1               2             5                                      3
2              26            39                                      2
3               0             0                                      4
4               0             0                                      2
5               0             1                                      3
6               0             0                                      4
  Ease.of.Onl

#### Here the customer ID will be just to represent the ID, so hence not considered. Therefore, dropping the ‘ID’ column from the dataset

In [4]:
%%R
df1=df[ ,-which(names(df) %in% c("ID"))]
head(df1)

  Gender Age Customer.Type Type.of.Travel    Class Flight.Distance
1   Male  48    First-time       Business Business             821
2 Female  35     Returning       Business Business             821
3   Male  41     Returning       Business Business             853
4   Male  50     Returning       Business Business            1905
5 Female  49     Returning       Business Business            3470
6   Male  43     Returning       Business Business            3788
  Departure.Delay Arrival.Delay Departure.and.Arrival.Time.Convenience
1               2             5                                      3
2              26            39                                      2
3               0             0                                      4
4               0             0                                      2
5               0             1                                      3
6               0             0                                      4
  Ease.of.Online.Booking Check.in.

### Label Encoding

#### Installing the superml packages

In [5]:
%%R 
#label encoding in R 
install.packages('superml')

(as ‘lib’ is unspecified)





























	‘/tmp/Rtmpma2UxD/downloaded_packages’



### Using a labelencoder, convert the satisfaction categorical column into 0 and 1 values.
#### Removing 'NA' values in the dataset
#### Storing the number of row value in the variable n.

In [6]:
%%R 
library('superml')
lbl = LabelEncoder$new()
df1$Satisfaction= lbl$fit_transform(df1$Satisfaction)
head(df1)

data=na.omit(df1)
n=nrow(data)
head(data)

  Gender Age Customer.Type Type.of.Travel    Class Flight.Distance
1   Male  48    First-time       Business Business             821
2 Female  35     Returning       Business Business             821
3   Male  41     Returning       Business Business             853
4   Male  50     Returning       Business Business            1905
5 Female  49     Returning       Business Business            3470
6   Male  43     Returning       Business Business            3788
  Departure.Delay Arrival.Delay Departure.and.Arrival.Time.Convenience
1               2             5                                      3
2              26            39                                      2
3               0             0                                      4
4               0             0                                      2
5               0             1                                      3
6               0             0                                      4
  Ease.of.Online.Booking Check.in.

#### Splitting the 80% of the data into train data and 20% of the data into test data

In [7]:
%%R
set.seed(134)
indexes=sample(n,n*(80/100)) 
trainset=data[indexes,]
testset=data[-indexes,]

#### Specify the significant variables on the output variable at the level of alpha=0.05

In [8]:
%%R
model=glm(Satisfaction~.,data=data,family="binomial") 
print(coef(model)) 
summary(model) 

                           (Intercept)                             GenderMale 
                         -7.813013e+00                           6.377047e-02 
                                   Age                 Customer.TypeReturning 
                         -8.489984e-03                           2.035678e+00 
                Type.of.TravelPersonal                           ClassEconomy 
                         -2.734543e+00                          -7.164590e-01 
                     ClassEconomy Plus                        Flight.Distance 
                         -8.158726e-01                          -1.725148e-05 
                       Departure.Delay                          Arrival.Delay 
                          4.234910e-03                          -9.117598e-03 
Departure.and.Arrival.Time.Convenience                 Ease.of.Online.Booking 
                         -1.329855e-01                          -1.514039e-01 
                      Check.in.Service              

#### Except Flight Distance , all other independent variables are significant. Since their p.value<alpha=0.05

#### Predict the output of the test dataset using the trained model

In [9]:
%%R
trainset.glm=glm(Satisfaction~.,trainset,family="binomial") # ~ sign indicates that we have included all the indipendent varibles.
res=predict(trainset.glm,testset, type="response")

In [10]:
%%R
predictedvalues=rep(0,length(res))
predictedvalues[res>0.5]=1 # this line indicates the probability of Satisfaction is 1, if the p<0.5 then Satisfaction=0

#### Confusion matrix and probability of correctness of predictions

In [11]:
%%R
actual=testset[,23]
confusion_matrix=table(predictedvalues, actualvalues=testset[,23])
confusion_matrix

               actualvalues
predictedvalues     0     1
              0 13267  1894
              1  1383  9354


#### True Negative(TN)=13267
#### False Positive(FP)=1894
#### False Negative(FN)=1383
#### True Positive(TP)=9354

In [12]:
%%R
accuracy=mean(predictedvalues==testset[,23]) #accuracy=(TN+TP)/n here n=13267+1894+1383+9354=25898
accuracy

[1] 0.8734651


#### Accuracy is 0.8734